<a href="https://colab.research.google.com/github/ehsan-lari/pyro101/blob/main/pyro_03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Bayesian Linear Regression

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.colors as mcolors
from scipy.stats import multivariate_normal, norm
import time
import torch
import torch.nn as nn
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import warnings
warnings.filterwarnings('ignore')

In [ ]:
DATA_URL = "https://github.com/pyro-ppl/brmp/raw/master/brmp/examples/rugged_data.csv"
data = pd.read_csv(DATA_URL, encoding="ISO-8859-1")
df = data[["cont_africa", "rugged", "rgdppc_2000"]]
df = df.sample(frac=1)
df = df[np.isfinite(df.rgdppc_2000)]
df["rgdppc_2000"] = np.log(df["rgdppc_2000"])
df = df[["cont_africa", "rugged", "rgdppc_2000"]]

In [ ]:
data = np.array(df)
x_data = {'non-african': data[data[:, 0] == 0, 1].reshape(-1,1), 'african': data[data[:, 0] == 1, 1].reshape(-1,1)}
y_data = {'non-african': data[data[:, 0] == 0, -1], 'african': data[data[:, 0] == 1, -1]}

print(x_data['african'].shape)
print(x_data['non-african'].shape)

### 1. Regression model

In [ ]:
class RegressionModel():
    def __init__(self):
        self.w = torch.nn.Parameter(torch.zeros(1, 1))
        self.b = torch.nn.Parameter(torch.zeros(1, 1))

    def params(self):
        return {"b": self.b, "w": self.w}

    def predict(self, x_data):
        return (self.b + torch.mm(self.w, torch.t(x_data))).squeeze(0)

In [ ]:
def least_squares_solution(x_data, y_data, verbose=True):
  regression_model = RegressionModel()
  loss_fn = nn.MSELoss(reduction='sum')
  optim = torch.optim.Adam(regression_model.params().values(), lr=1e-1)
  NUM_ITERATIONS = 1000

  param = {}

  for cont in x_data.keys():
    param[cont] = {}
    if verbose:
      print(f"Learning model for {cont} nations")
    for iter in range(NUM_ITERATIONS):
      y_pred = regression_model.predict(torch.tensor(x_data[cont], dtype=torch.float))
      loss = loss_fn(y_pred, torch.tensor(y_data[cont], dtype=torch.float))
      optim.zero_grad()
      loss.backward()
      optim.step()
    param[cont]['w'] = regression_model.params()['w'].item()
    param[cont]['b'] = regression_model.params()['b'].item()

    if verbose:
      print(f"Learned parameters:")
      for cont in param.keys():
        print(f"{cont}: w = {param[cont]['w']:.4f}, b = {param[cont]['b']:.4f}")
  return param

In [ ]:
model_lr = least_squares_solution(x_data, y_data)

### 2. Bayesian linear regression model

#### 2.1 Calculate ELBO

In [ ]:
def calculate_ELBO(x_data, y_data, gamma_w, gamma_b, theta, q_w_mean, q_w_prec, q_b_mean, q_b_prec):

  M = x_data.shape[1]

  E_log_p = -.5 * M * np.log( (2 * np.pi)) + .5 * M * np.log( gamma_w ) \
            -.5 * gamma_w * np.sum( np.diag(np.linalg.inv(q_w_prec)) + (q_w_mean**2).flatten() )

  E_log_p += -.5 * np.log( (2 * np.pi)) + .5 * np.log(gamma_b) -.5 * gamma_b * ( 1/q_b_prec + q_b_mean**2)

  E_w_w = np.linalg.inv(q_w_prec) + q_w_mean @ q_w_mean.T
  E_b_b = 1/q_b_prec + q_b_mean**2

  for i in range(x_data.shape[0]):
    E_x_ww_x = np.matmul(np.transpose(x_data[i, :]), E_w_w @ x_data[i, :])
    E_log_p += -.5 * np.log((2 * np.pi)) + .5 * np.log(theta) \
               -.5 * theta * (y_data[i]**2 + E_x_ww_x + E_b_b
                              + 2 * q_b_mean * q_w_mean.T @ x_data[i, :]
                              - 2 * y_data[i] * q_w_mean.T @ x_data[i, :]
                              - 2 * y_data[i] * q_b_mean)

  ent = .5 * np.log( (np.pi) * np.exp(1) / q_b_prec )
  ent += .5 * np.log(np.linalg.det( 2 * np.pi * np.exp(1) * np.linalg.inv(q_w_prec) ) )

  return E_log_p - ent

#### 2.2 Updating equations

In [ ]:
def update_w(x_data, y_data, gamma_w, theta, q_w_mean, q_w_prec, q_b_mean, comp):
  M = x_data.shape[1]
  tau = gamma_w
  mu = 0.0

  for i in range(x_data.shape[0]):
    tau += theta * x_data[i, comp]**2
    mu += (y_data[i] * q_b_mean - (np.sum(x_data[i, :] @ q_w_mean ) - x_data[i, comp] * q_w_mean[comp] ) ) * x_data[i, comp]

  mu = theta * 1/tau * mu

  q_w_prec[comp, comp] = tau
  q_w_mean[comp] = mu.item()

  return q_w_mean, q_w_prec

def update_b(x_data, y_data, gamma_b, theta, q_w_mean):
  tau = (gamma_b + theta * x_data.shape[0])
  mu = 0.0

  for i in range(x_data.shape[0]):
    mu += (y_data[i] - q_w_mean.T @ x_data[i, :])

  mu = theta * 1/tau * mu

  return tau, mu

In [ ]:
def variational_solution(x_data, y_data, gamma_w: float = 1, gamma_b: float = 1, theta: float = 1, verbose=True):
    """
    Perform variational inference using the coordinate ascent updating rules. The two models (for african and
    non-african nations are learned independently.
    :param x_data: dictionary with data for the predictor variable
    :param y_data: dictionary with data for the response variable
    :param gamma_w: precision for the weight/slope term
    :param gamma_b: precision for the bias term
    :param theta: prior precision for y
    :param verbose: Do we want to display information from the updating process?
    :return: The model (i.e., mean and covariance) in a dictionary with one element for african and non-african.
    """
    param = {}

    if verbose:
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), sharey=True)
        fig.suptitle("Evolution of ELBO", fontsize=16)

     # Do the learning for african and non-african nations
    for idx, cont in enumerate(x_data):
        param[cont] = {}
        M = x_data[cont].shape[1]
        q_w_mean = np.random.normal(0, 1, (1, 1))
        q_w_prec = np.array([[1]])
        q_b_mean = np.random.normal(0, 1)
        q_b_prec = 1
        elbos = []

        # Calculate ELBO
        this_lb = calculate_ELBO(x_data[cont], y_data[cont], gamma_w, gamma_b, theta, q_w_mean, q_w_prec, q_b_mean, q_b_prec)
        elbos.append(this_lb)
        previous_lb = -np.inf
        # Start iterating
        if verbose:
            print("\n" + 100 * "=" + "\n   VB iterations:\n" + 100 * "=")
        for iteration in range(100):

            # Update the variational distributions
            for i in range(M):
                q_w_prec, q_w_mean = update_w(x_data[cont], y_data[cont], gamma_w, theta, q_w_mean, q_w_prec, q_b_mean, i)
            q_b_prec, q_b_mean = update_b(x_data[cont], y_data[cont], gamma_b, theta, q_w_mean)

            this_lb = calculate_ELBO(x_data[cont], y_data[cont], gamma_w, gamma_b, theta, q_w_mean, q_w_prec, q_b_mean, q_b_prec)
            elbos.append(this_lb)
            if verbose:
                print(f"Iteration {iteration:2d}. ELBO: {this_lb.item():13.7f}")
            if this_lb < previous_lb:
                raise ValueError("ELBO is decreasing. Something is wrong! Goodbye...")

            if iteration > 0 and np.abs((this_lb - previous_lb) / previous_lb) < 1E-8:
                # Very little improvement. We are done.
                break

            # If we didn't break we need to run again. Update the value for "previous"
            previous_lb = this_lb
        if verbose:
            print("\n" + 100 * "=" + "\n")

        param[cont]['w'] = np.r_[q_b_mean, q_w_mean.flatten()]
        param[cont]['cov'] = np.linalg.inv(np.diag(np.r_[q_b_prec, q_w_prec.flatten()]))

        if verbose:
            ax[idx].plot(range(len(elbos)), elbos, label='Mean field approximation: ELBO')
            ax[idx].set(xlabel="Number of iterations",
                        ylabel="ELBO",
                        title=f"{cont} nations")
            ax[idx].legend()

    plt.show()

    # Inspect learned parameters
    if verbose:
        print("Learned parameters:")
        for cont in param.keys():
            print(f"For {cont} nations")
            for name, value in param[cont].items():
                print(f"{name} = ")
                print(f"{value}")

    return param

In [ ]:
model_mf = variational_solution(x_data, y_data, gamma_w=1, gamma_b=1, theta=1)